# Playground for Clustering Slides

- Originally by Stephen W. Thomas
- Modified by Ahmed E. Hassan
- Used for MMA 869 and MMAI 869

## ALWAYS: timestamp your notebook so you have an idea of when it was last executed

In [1]:
import datetime
print(datetime.datetime.now())

2019-11-24 18:19:02.288240


# Import needed libraries
- yellowbrick, pandas_profiling and KModes are usually not installed by default
- Install anaconda.org at https://www.anaconda.com/distribution/#download-section (Go with python 3.+)
- Once installed Launch  "Anaconda Navigator" then
- Environments => base(root) => PLAY symbol => Open Terminal => "pip install pandas-profiling"


In [5]:
import pandas as pd
import numpy as np

import matplotlib as mpl
mpl.rcParams.update({'figure.max_open_warning': 0})
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.metrics import silhouette_score, silhouette_samples
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

import pandas_profiling

import itertools

import scipy

from yellowbrick.cluster import SilhouetteVisualizer, InterclusterDistance, KElbowVisualizer

from kmodes.kmodes import KModes

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# This will ensure that matplotlib figures don't get cut off when saving with savefig()
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# Read in Data and Examine the Profile of the DAta

In [6]:
#df = pd.read_csv("https://raw.githubusercontent.com/stepthom/sandbox/master/data/Mall_Customers.csv")
df = pd.read_csv("smalljoin.csv")

list(df)
df.shape
df.info()
df.describe().transpose()
df.head(n=20)
df.tail()



['Unnamed: 0',
 'Totals_Table_Customer_ID',
 'TotalVisits',
 'TotalWeekendVisits',
 'TotalWeekDayVisits',
 'Total_Dwell_TIme',
 'Weekend_Dwell_TIme',
 'WeekDay_Dwell_TIme',
 'Average_Dwell_Time',
 'BankDaysVisited',
 'Total_Bank_Dwell_Time',
 'Avg_Bank_Dwell_Time',
 'min_Bank_Dwell_time',
 'max_Bank_Dwell_time',
 'Children.and.Infants.ApparelDaysVisited',
 'Total_Children.and.Infants.Apparel_Dwell_Time',
 'Avg_Children.and.Infants.Apparel_Dwell_Time',
 'min_Children.and.Infants.Apparel_Dwell_time',
 'max_Children.and.Infants.Apparel_Dwell_time',
 'CorridorDaysVisited',
 'Total_Corridor_Dwell_Time',
 'Avg_Corridor_Dwell_Time',
 'min_Corridor_Dwell_time',
 'max_Corridor_Dwell_time',
 'DiningTimesVisited',
 'Total_Dining_Dwell_Time',
 'Avg_Dining_Dwell_Time',
 'min_Dining_Dwell_time',
 'max_Dining_Dwell_time',
 'ElectronicsDaysVisited',
 'Total_Electronics_Dwell_Time',
 'Avg_Electronics_Dwell_Time',
 'min_Electronics_Dwell_time',
 'max_Electronics_Dwell_time',
 'EntranceDaysVisited',
 'To

(100547, 154)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100547 entries, 0 to 100546
Columns: 154 entries, Unnamed: 0 to max_Womens.Apparel_Dwell_time
dtypes: float64(29), int64(125)
memory usage: 118.1 MB


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,100547.0,331989.568759,210966.545871,2.0,146070.5,317077.0,508147.5,733074.0
Totals_Table_Customer_ID,100547.0,331990.914707,210966.992326,2.0,146071.5,317078.0,508148.5,733077.0
TotalVisits,100547.0,2.449044,3.118432,1.0,1.0,1.0,3.0,78.0
TotalWeekendVisits,100547.0,0.901200,1.305816,0.0,0.0,1.0,1.0,27.0
TotalWeekDayVisits,100547.0,1.547843,2.309494,0.0,0.0,1.0,2.0,59.0
...,...,...,...,...,...,...,...,...
Womens.ApparelDaysVisited,100547.0,0.424289,0.749982,0.0,0.0,0.0,1.0,38.0
Total_Womens.Apparel_Dwell_Time,100547.0,8.345629,41.880457,0.0,0.0,0.0,6.0,7364.0
Avg_Womens.Apparel_Dwell_Time,100547.0,6.176976,15.892810,0.0,0.0,0.0,5.0,368.2
min_Womens.Apparel_Dwell_time,100547.0,5.472635,14.863210,0.0,0.0,0.0,5.0,341.0


,Unnamed: 0,Totals_Table_Customer_ID,TotalVisits,TotalWeekendVisits,TotalWeekDayVisits,Total_Dwell_TIme,Weekend_Dwell_TIme,WeekDay_Dwell_TIme,Average_Dwell_Time,BankDaysVisited,...,WashroomDaysVisited,Total_Washroom_Dwell_Time,Avg_Washroom_Dwell_Time,min_Washroom_Dwell_time,max_Washroom_Dwell_time,Womens.ApparelDaysVisited,Total_Womens.Apparel_Dwell_Time,Avg_Womens.Apparel_Dwell_Time,min_Womens.Apparel_Dwell_time,max_Womens.Apparel_Dwell_time
0,2,2,4,0,4,333,0,333,83.250000,0,...,0,0,0.0,0,0,0,0,0.000000,0,0
1,7,7,2,0,2,939,0,939,469.500000,0,...,0,0,0.0,0,0,1,76,76.000000,76,76
2,22,22,4,2,2,1473,917,556,368.250000,0,...,0,0,0.0,0,0,2,37,18.500000,8,29
3,28,28,4,1,3,1032,231,801,258.000000,0,...,0,0,0.0,0,0,1,11,11.000000,11,11
4,34,34,1,0,1,61,0,61,61.000000,0,...,0,0,0.0,0,0,0,0,0.000000,0,0
5,40,40,6,5,1,1179,1156,23,196.500000,0,...,0,0,0.0,0,0,1,13,13.000000,13,13
6,41,41,5,1,4,1265,480,785,253.000000,0,...,1,6,6.0,6,6,1,5,5.000000,5,5
7,42,42,20,7,13,2956,1039,1917,147.800000,0,...,1,3,3.0,3,3,7,66,9.428571,4,17
8,76,76,2,0,2,399,0,399,199.500000,0,...,0,0,0.0,0,0,0,0,0.000000,0,0
9,92,92,5,3,2,1005,488,517,201.000000,0,...,1,6,6.0,6,6,3,21,7.000000,4,12


,Unnamed: 0,Totals_Table_Customer_ID,TotalVisits,TotalWeekendVisits,TotalWeekDayVisits,Total_Dwell_TIme,Weekend_Dwell_TIme,WeekDay_Dwell_TIme,Average_Dwell_Time,BankDaysVisited,...,WashroomDaysVisited,Total_Washroom_Dwell_Time,Avg_Washroom_Dwell_Time,min_Washroom_Dwell_time,max_Washroom_Dwell_time,Womens.ApparelDaysVisited,Total_Womens.Apparel_Dwell_Time,Avg_Womens.Apparel_Dwell_Time,min_Womens.Apparel_Dwell_time,max_Womens.Apparel_Dwell_time
100542,733015,733018,1,1,0,239,239,0,239.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100543,733041,733044,1,1,0,456,456,0,456.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100544,733058,733061,1,1,0,172,172,0,172.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100545,733071,733074,1,1,0,146,146,0,146.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100546,733074,733077,1,1,0,230,230,0,230.0,0,...,1,22,22.0,22,22,0,0,0.0,0,0


# Plot the two metrics that are used for clustering

In [7]:
import matplotlib.pyplot as plt
 


# Plot the two clustering metrics using log scale to cope with the varying scales

In [ ]:
import pandas as pd;



# Scale both clustering metrics to be on the same scale

In [12]:
X = df.copy()



df.head()
Y = X.drop(['Totals_Table_Customer_ID', 'Unnamed: 0'], axis=1) #axis 0 to drop rows

Z = Y.drop(['min_Washroom_Dwell_time','max_Washroom_Dwell_time','min_Womens.Apparel_Dwell_time','max_Womens.Apparel_Dwell_time','min_Bank_Dwell_time','max_Bank_Dwell_time','min_Children.and.Infants.Apparel_Dwell_time', 'max_Children.and.Infants.Apparel_Dwell_time','min_Corridor_Dwell_time', 'max_Corridor_Dwell_time','min_Dining_Dwell_time','max_Dining_Dwell_time','min_Electronics_Dwell_time', 'max_Electronics_Dwell_time','min_Entrance_Dwell_time','max_Entrance_Dwell_time', 'min_Family.Apparel_Dwell_time', 'max_Family.Apparel_Dwell_time', 'min_Fashion.Accessories_Dwell_time', 'max_Fashion.Accessories_Dwell_time','min_Food.Court_Dwell_time', 'max_Food.Court_Dwell_time','min_Food.Court_Dwell_time', 'max_Food.Court_Dwell_time','min_Footwear_Dwell_time','max_Footwear_Dwell_time','min_Full.Service.Restaurants_Dwell_time', 'max_Full.Service.Restaurants_Dwell_time','min_Furniture...Home.Furnishings_Dwell_time', 'max_Furniture...Home.Furnishings_Dwell_time','min_General.merch.and.variety.stores_Dwell_time', 'max_General.merch.and.variety.stores_Dwell_time','min_Health.and.Beauty_Dwell_time', 'max_Health.and.Beauty_Dwell_time','min_Jewellery_Dwell_time', 'max_Jewellery_Dwell_time','min_Large.Format_Dwell_time', 'max_Large.Format_Dwell_time','min_Lux_Dwell_time', 'max_Lux_Dwell_time','min_Mens.Apparel_Dwell_time', 'max_Mens.Apparel_Dwell_time','min_Non.Food.Court.Fast.Food_Dwell_time', 'max_Non.Food.Court.Fast.Food_Dwell_time','min_Office.supply.stationary.gifts_Dwell_time', 'max_Office.supply.stationary.gifts_Dwell_time','min_Other.Services_Dwell_time', 'max_Other.Services_Dwell_time','min_Personal.Care.Services_Dwell_time', 'max_Personal.Care.Services_Dwell_time','min_Recreation_Dwell_time', 'max_Recreation_Dwell_time','min_Specialty.Apparel_Dwell_time', 'max_Specialty.Apparel_Dwell_time','min_Specialty.Food.Stores_Dwell_time', 'max_Specialty.Food.Stores_Dwell_time','min_Specialty.Leasing_Dwell_time', 'max_Specialty.Leasing_Dwell_time','min_Unassigned_Dwell_time', 'max_Unassigned_Dwell_time'], axis=1)

JustLux = Z[['TotalVisits', 'TotalWeekendVisits', 'TotalWeekDayVisits', 'Total_Dwell_TIme', 'LuxuryTimesVisited','Total_Luxury_Dwell_Time','DiningTimesVisited','Total_Dining_Dwell_Time','Family.ApparelDaysVisited','Total_Family.Apparel_Dwell_Time','Fashion.AccessoriesDaysVisited','Total_Fashion.Accessories_Dwell_Time','FootwearDaysVisited','Total_Footwear_Dwell_Time', 'JewelleryDaysVisited','Total_Jewellery_Dwell_Time','Large.FormatDaysVisited','Total_Large.Format_Dwell_Time','Mens.ApparelDaysVisited','Total_Mens.Apparel_Dwell_Time','RecreationDaysVisited','Total_Recreation_Dwell_Time','Specialty.LeasingDaysVisited','Total_Specialty.Leasing_Dwell_Time','Womens.ApparelDaysVisited','Total_Womens.Apparel_Dwell_Time','ElectronicsDaysVisited','Total_Electronics_Dwell_Time','Full.Service.RestaurantsDaysVisited','Total_Full.Service.Restaurants_Dwell_Time','Total_Specialty.Apparel_Dwell_Time','Avg_Specialty.Apparel_Dwell_Time','Health.and.BeautyDaysVisited','Total_Health.and.Beauty_Dwell_Time','Specialty.Food.StoresDaysVisited','Total_Specialty.Food.Stores_Dwell_Time','Children.and.Infants.ApparelDaysVisited','Total_Children.and.Infants.Apparel_Dwell_Time']].copy()



#scaler = StandardScaler()

#Z = scaler.fit_transform(Z)

#box_plot_data=[X["AnnualIncome"],X["SpendingScore"]]
#plt.boxplot(box_plot_data,labels=lb, vert=0)
#plt.show()

#Z = Z.sample(frac=0.4, random_state=1)

,Unnamed: 0,Totals_Table_Customer_ID,TotalVisits,TotalWeekendVisits,TotalWeekDayVisits,Total_Dwell_TIme,Weekend_Dwell_TIme,WeekDay_Dwell_TIme,Average_Dwell_Time,BankDaysVisited,...,WashroomDaysVisited,Total_Washroom_Dwell_Time,Avg_Washroom_Dwell_Time,min_Washroom_Dwell_time,max_Washroom_Dwell_time,Womens.ApparelDaysVisited,Total_Womens.Apparel_Dwell_Time,Avg_Womens.Apparel_Dwell_Time,min_Womens.Apparel_Dwell_time,max_Womens.Apparel_Dwell_time
0,2,2,4,0,4,333,0,333,83.25,0,...,0,0,0.0,0,0,0,0,0.0,0,0
1,7,7,2,0,2,939,0,939,469.50,0,...,0,0,0.0,0,0,1,76,76.0,76,76
2,22,22,4,2,2,1473,917,556,368.25,0,...,0,0,0.0,0,0,2,37,18.5,8,29
3,28,28,4,1,3,1032,231,801,258.00,0,...,0,0,0.0,0,0,1,11,11.0,11,11
4,34,34,1,0,1,61,0,61,61.00,0,...,0,0,0.0,0,0,0,0,0.0,0,0


In [13]:
JustLux.shape
#X.info()
#X.describe()
#X.describe().transpose()
JustLux.head(10)
X.head(10).transpose()
X.tail()

(100547, 38)

,TotalVisits,TotalWeekendVisits,TotalWeekDayVisits,Total_Dwell_TIme,LuxuryTimesVisited,Total_Luxury_Dwell_Time,DiningTimesVisited,Total_Dining_Dwell_Time,Family.ApparelDaysVisited,Total_Family.Apparel_Dwell_Time,...,Full.Service.RestaurantsDaysVisited,Total_Full.Service.Restaurants_Dwell_Time,Total_Specialty.Apparel_Dwell_Time,Avg_Specialty.Apparel_Dwell_Time,Health.and.BeautyDaysVisited,Total_Health.and.Beauty_Dwell_Time,Specialty.Food.StoresDaysVisited,Total_Specialty.Food.Stores_Dwell_Time,Children.and.Infants.ApparelDaysVisited,Total_Children.and.Infants.Apparel_Dwell_Time
0,4,0,4,333,1,55,0,0,0,0,...,0,0,0,0.0,1,5,0,0,0,0
1,2,0,2,939,1,48,1,52,2,34,...,0,0,4,4.0,0,0,0,0,0,0
2,4,2,2,1473,1,10,2,62,1,13,...,0,0,10,10.0,0,0,0,0,0,0
3,4,1,3,1032,2,47,3,134,3,43,...,1,71,6,6.0,2,12,0,0,1,3
4,1,0,1,61,1,19,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0,0
5,6,5,1,1179,1,10,2,40,1,25,...,1,130,0,0.0,1,14,0,0,0,0
6,5,1,4,1265,2,142,0,0,1,3,...,0,0,14,14.0,0,0,1,20,0,0
7,20,7,13,2956,3,80,14,363,3,50,...,5,203,135,27.0,3,27,2,18,1,12
8,2,0,2,399,1,17,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0,0
9,5,3,2,1005,2,21,1,6,4,27,...,0,0,0,0.0,1,3,0,0,0,0


,0,1,2,3,4,5,6,7,8,9
Unnamed: 0,2.0,7.0,22.0,28.0,34.0,40.0,41.0,42.000000,76.0,92.0
Totals_Table_Customer_ID,2.0,7.0,22.0,28.0,34.0,40.0,41.0,42.000000,76.0,92.0
TotalVisits,4.0,2.0,4.0,4.0,1.0,6.0,5.0,20.000000,2.0,5.0
TotalWeekendVisits,0.0,0.0,2.0,1.0,0.0,5.0,1.0,7.000000,0.0,3.0
TotalWeekDayVisits,4.0,2.0,2.0,3.0,1.0,1.0,4.0,13.000000,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
Womens.ApparelDaysVisited,0.0,1.0,2.0,1.0,0.0,1.0,1.0,7.000000,0.0,3.0
Total_Womens.Apparel_Dwell_Time,0.0,76.0,37.0,11.0,0.0,13.0,5.0,66.000000,0.0,21.0
Avg_Womens.Apparel_Dwell_Time,0.0,76.0,18.5,11.0,0.0,13.0,5.0,9.428571,0.0,7.0
min_Womens.Apparel_Dwell_time,0.0,76.0,8.0,11.0,0.0,13.0,5.0,4.000000,0.0,4.0


,Unnamed: 0,Totals_Table_Customer_ID,TotalVisits,TotalWeekendVisits,TotalWeekDayVisits,Total_Dwell_TIme,Weekend_Dwell_TIme,WeekDay_Dwell_TIme,Average_Dwell_Time,BankDaysVisited,...,WashroomDaysVisited,Total_Washroom_Dwell_Time,Avg_Washroom_Dwell_Time,min_Washroom_Dwell_time,max_Washroom_Dwell_time,Womens.ApparelDaysVisited,Total_Womens.Apparel_Dwell_Time,Avg_Womens.Apparel_Dwell_Time,min_Womens.Apparel_Dwell_time,max_Womens.Apparel_Dwell_time
100542,733015,733018,1,1,0,239,239,0,239.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100543,733041,733044,1,1,0,456,456,0,456.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100544,733058,733061,1,1,0,172,172,0,172.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100545,733071,733074,1,1,0,146,146,0,146.0,0,...,0,0,0.0,0,0,0,0,0.0,0,0
100546,733074,733077,1,1,0,230,230,0,230.0,0,...,1,22,22.0,22,22,0,0,0.0,0,0


## Plot the Data

# K-Means

In [ ]:
k_means = KMeans(init='k-means++', n_clusters=20, n_init=10, random_state=42, n_jobs=-1) #### n_jobs 
#k_means = KMeans(init='random', n_clusters=5, n_init=10, random_state=42)
#k_means.fit(X)

k_means.fit(JustLux)

In [7]:
k_means.labels_

array([ 4,  0, 13, ..., 17, 17, 17])

In [8]:
# Let's look at the centers
k_means.cluster_centers_

array([[1.15302338e+00, 4.13158094e-01, 7.39865291e-01, 3.41870790e+01,
        3.07709174e+01, 5.30259075e-02, 5.31671754e-01, 5.26929189e-01],
       [4.39496589e+00, 1.60479887e+00, 2.79016702e+00, 9.43944248e+02,
        2.76241849e+02, 8.09927076e-01, 1.96758410e+01, 1.33517878e+01],
       [2.08543046e+01, 6.68874172e+00, 1.41655629e+01, 5.22401987e+03,
        3.11374004e+02, 3.54304636e+00, 1.52867550e+02, 2.92952249e+01],
       [1.30553097e+01, 4.04203540e+00, 9.01327434e+00, 2.95443584e+03,
        3.08457985e+02, 2.24336283e+00, 6.71327434e+01, 2.19626697e+01],
       [1.00000000e+00, 3.85335757e-01, 6.14664243e-01, 2.09530711e+02,
        2.09530711e+02, 2.05255330e-01, 4.80917755e+00, 4.80917755e+00],
       [1.04738124e+01, 3.39585871e+00, 7.07795371e+00, 2.20531303e+03,
        2.86573782e+02, 1.82947625e+00, 5.00986602e+01, 1.82479313e+01],
       [3.52200000e+01, 1.13200000e+01, 2.39000000e+01, 9.55092000e+03,
        3.23819835e+02, 4.46000000e+00, 1.92340000e+02, 2.

# Plot the Clusters

In [ ]:
plt.style.use('default');

plt.figure(figsize=(16, 10));
plt.grid(True);

sc = plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=200, c=k_means.labels_);
# plt.scatter(k_means.cluster_centers_[:, 0], k_means.cluster_centers_[:, 1], marker='x', s=500, c="black")
plt.title("K-Means (K=5)", fontsize=20);
plt.xlabel('Annual Income (K)', fontsize=22);
plt.ylabel('Spending Score', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);


#for label in k_means.labels_:
#    plt.text(x=k_means.cluster_centers_[label, 0], y=k_means.cluster_centers_[label, 1], s=label, fontsize=32, 
#             horizontalalignment='center', verticalalignment='center', color='black',
#             bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1', alpha=0.02));4

def PrintLabels (k_means):
    for label in k_means.labels_:
        plt.text(x=k_means.cluster_centers_[label, 0], y=k_means.cluster_centers_[label, 1], s=label, fontsize=32, 
                 horizontalalignment='center', verticalalignment='center', color='black',
                 bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1', alpha=0.02));
    return

PrintLabels (k_means)

#plt.savefig('out/mall-kmeans-5.png');

## Internal Validation Metrics

In [9]:
# WCSS == Inertia
k_means.inertia_

1062518515.4467034

In [15]:
silhouette_score(JustLux, k_means.labels_)

0.4800654184222223

In [ ]:
plt.style.use('default');

sample_silhouette_values = silhouette_samples(X, k_means.labels_)
import math;
sizes =200*sample_silhouette_values

plt.figure(figsize=(16, 10));
plt.grid(True);

plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=sizes, c=k_means.labels_)
plt.scatter(k_means.cluster_centers_[:, 0], k_means.cluster_centers_[:, 1], marker='x', s=500, c="black")

plt.title("K-Means (Dot Size = Silhouette Distance)", fontsize=20);
plt.xlabel('Annual Income (K)', fontsize=22);
plt.ylabel('Spending Score', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);

#plt.savefig('out/mall-kmeans-5-silhouette-size.png');

In [ ]:
#sklearn.metrics.davies_bouldin_score(X, k_means.labels_)

In [ ]:
visualizer = SilhouetteVisualizer(k_means)
visualizer.fit(X)
visualizer.poof()
fig = visualizer.ax.get_figure()
#fig.savefig('out/mall-kmeans-5-silhouette.png', transparent=False);

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = InterclusterDistance(k_means)

visualizer.fit(X) # Fit the training data to the visualizer
visualizer.poof() # Draw/show/poof the data
#plt.savefig('out/mall-kmeans-5-tsne.png', transparent=False);

## Elbow Method (Manual)

In [17]:
inertias = {}
silhouettes = {}
for k in range(5, 10):
    kmeans = KMeans(init='k-means++', n_init=10, n_clusters=k, max_iter=1000, random_state=42).fit(Z)
    inertias[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
    silhouettes[k] = silhouette_score(JustLux, kmeans.labels_, metric='euclidean')
    

plt.figure();
plt.grid(True);
plt.plot(list(inertias.keys()), list(inertias.values()));
plt.title('K-Means, Elbow Method')
plt.xlabel("Number of clusters, K");
plt.ylabel("Inertia");
#plt.savefig('out/mall-kmeans-elbow-interia.png');


plt.figure();
plt.grid(True);
plt.plot(list(silhouettes.keys()), list(silhouettes.values()));
plt.title('K-Means, Elbow Method')
plt.xlabel("Number of clusters, K");
plt.ylabel("Silhouette");
#plt.savefig('out/mall-kmeans-elbow-silhouette.png');

MemoryError: 

## Elbow Method (Using Yellowbrick Package)

In [ ]:
model = KMeans(init='k-means++', n_init=10, max_iter=1000, random_state=42)
KElbowVisualizer(model, k=(2,11), metric='silhouette', timings=False).fit(Z).poof();
#KElbowVisualizer(model, k=(2,11), metric='calinski_harabaz', timings=False).fit(X).poof();
KElbowVisualizer(model, k=(2,11), metric='distortion', timings=False).fit(Z).poof();

## Intepretting the Clusters

### Means

In [ ]:
k_means.cluster_centers_

In [ ]:
for label in set(k_means.labels_):
    print('\nCluster {}:'.format(label))
    X_tmp = X[k_means.labels_==label].copy()
    X_tmp.loc['mean'] = X_tmp.mean()
    X_tmp.tail(13)
    

# Find Examplars

Let's find the observation that is closest to the cluster center

In [ ]:
from scipy.spatial import distance

for i, label in enumerate(set(k_means.labels_)):    
    X_tmp = X[k_means.labels_==label].copy()
    
    exemplar_idx = distance.cdist([k_means.cluster_centers_[i]], X_tmp).argmin()
    exemplar = pd.DataFrame(X_tmp.iloc[exemplar_idx])
   
    print('\nCluster {}:'.format(label))
    exemplar.transpose()

### Snake Plot

In [ ]:
dat = X.copy()

dat['Cluster'] = k_means.labels_

datamart_melt = pd.melt(dat.reset_index(),
id_vars=['Cluster'],
value_vars=['AnnualIncome','SpendingScore'],
var_name='Feature',
value_name='Value')

plt.title('Snake Plot, K-Means, K=5')
sns.lineplot(x="Feature", y="Value", hue='Cluster', data=datamart_melt)
plt.savefig('out/mall-kmeans-5-snake.png', transparent=False);

# Relative Importance Plots

In [ ]:
cluster_avg = dat.groupby(['Cluster']).mean()
population_avg = dat.drop(['Cluster'], axis=1).mean()

relative_imp = cluster_avg - population_avg

plt.figure(figsize=(8, 2));
plt.title('Relative importance of features');
sns.heatmap(data=relative_imp, annot=True, fmt='.2f', cmap='RdYlGn');
plt.savefig('out/mall-kmeans-5-importance.png', transparent=False);

## Experiment with Lots of Ks

In [ ]:
def do_kmeans(X, k):
    k_means = KMeans(init='k-means++', n_clusters=k, n_init=10, random_state=42)
    k_means.fit(X)
    wcss = k_means.inertia_
    sil = silhouette_score(X, k_means.labels_)
    
    plt.style.use('default');

    sample_silhouette_values = silhouette_samples(X, k_means.labels_)
    sizes = 200*sample_silhouette_values

    plt.figure(figsize=(16, 10));
    plt.grid(True);

    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=sizes, c=k_means.labels_)
    plt.scatter(k_means.cluster_centers_[:, 0], k_means.cluster_centers_[:, 1], marker='x', s=500, c="black")

    plt.title("K-Means (K={}, WCSS={:.2f}, Sil={:.2f})".format(k, wcss, sil), fontsize=20);
    plt.xlabel('Annual Income (K)', fontsize=22);
    plt.ylabel('Spending Score', fontsize=22);
    plt.xticks(fontsize=18);
    plt.yticks(fontsize=18);
    plt.savefig('out/mall-kmeans-auto-{}-silhouette-size.png'.format(k));
    plt.show()
    
    
    visualizer = SilhouetteVisualizer(k_means)
    visualizer.fit(X)
    visualizer.poof()
    fig = visualizer.ax.get_figure()
    fig.savefig('out/mall-kmeans-auto-{}-silhouette-plot.png'.format(k), transparent=False);
    
    print("K={}, WCSS={:.2f}, Sil={:.2f}".format(k, wcss, sil))

for k in range(2, 21):
    do_kmeans(Z, k)

# Look at Individual Silhouette Scores

In [ ]:
k_means = KMeans(init='k-means++', n_clusters=7, n_init=10, random_state=42)
k_means.fit(Z)
sample_silhouette_values = silhouette_samples(Z, k_means.labels_)

X_tmp = Z.copy()
X_tmp['Cluster ID'] = k_means.labels_

In [ ]:
X_tmp['Silhouette'] = sample_silhouette_values
X_tmp = X_tmp.sort_values(['Silhouette'])
X_tmp.head()
X_tmp.tail()

In [ ]:
plt.figure(figsize=(16, 10));
plt.grid(True);

plt.scatter(X_tmp['AnnualIncome'], X_tmp['SpendingScore'], sizes=200*(X_tmp['Silhouette']+0.05), c=X_tmp['Cluster ID'])

plt.title("K-Means (K = {}, Sil={:.2f})".format(k_means.n_clusters, silhouette_score(X, k_means.labels_, metric='euclidean')), fontsize=20);
plt.xlabel('Annual Income (K)', fontsize=22);
plt.ylabel('Spending Score', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);

#for label in k_means.labels_:
    #plt.text(x=k_means.cluster_centers_[label, 0], y=k_means.cluster_centers_[label, 1], s=label, fontsize=32, 
             #horizontalalignment='center', verticalalignment='center', color='black',
             #bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1', alpha=0.02));
    
#for i, txt in enumerate(X_tmp.index.values):
    #sil = X_tmp['Silhouette'].iloc[i]
    #if sil < 0.05:
        #plt.text(x=X_tmp['AnnualIncome'].iloc[i], y=X_tmp['SpendingScore'].iloc[i], s=txt, fontsize=22)

#plt.savefig('out/mall-kmeans-{}-silhouette-size-labels.png'.format(k_means.n_clusters));

In [ ]:
visualizer = SilhouetteVisualizer(k_means)
visualizer.fit(X)
visualizer.poof()
fig = visualizer.ax.get_figure()
fig.savefig('out/mall-kmeans-{}-silhouette-f.png'.format(k_means.n_clusters), transparent=False);

# DBSCAN

In [11]:
db = DBSCAN(eps=0.3, min_samples=2)
db.fit(JustLux)

DBSCAN(algorithm='auto', eps=0.3, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=2, n_jobs=None, p=None)

In [12]:
db.labels_

array([   0,    1,   -1, ...,  199,  293, 1748], dtype=int64)

In [14]:
silhouette_score(JustLux, db.labels_)

0.5522676130362911

In [ ]:
plt.figure(figsize=(16, 10));
plt.grid(True);

unique_labels = set(db.labels_)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))];

for k in unique_labels:
    if k == -1:        # Black used for noise.
        col = [0, 0, 0, 1]
    else:
        col = colors[k]

    xy = X[db.labels_ == k]
    plt.plot(xy.iloc[:, 0], xy.iloc[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=14);

    
plt.title('');
plt.title("DBSCAN (n_clusters = {:d}, black = outliers)".format(len(unique_labels)), fontsize=20);
plt.xlabel('Annual Income (K)', fontsize=22);
plt.ylabel('Spending Score', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
#plt.savefig('out/mall-dbscan-03.png', transparent=False);

## Elbow Method

In [ ]:
silhouettes = {}
for eps in np.arange(0.1, 0.6, 0.1):
    db = DBSCAN(eps=eps, min_samples=3).fit(Z)
    silhouettes[eps] = silhouette_score(Z, db.labels_, metric='euclidean')
    

plt.figure();
plt.plot(list(silhouettes.keys()), list(silhouettes.values()));
plt.title('DBSCAN, Elbow Method')
plt.xlabel("Eps");
plt.ylabel("Silhouette");
plt.grid(True);
#plt.savefig('out/mall-dbscan-03-silhouette.png', transparent=False);

## Parameter Exploration

In [ ]:
def do_dbscan(X, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)

    unique_labels = set(db.labels_)
    n_clusters = len(unique_labels)
    
    if n_clusters <= 1:
        print('eps={}, min_samples={}, n_clusters <= 1. Returning.'.format(eps, min_samples))
        return
    
    sil = silhouette_score(X, db.labels_)
    print("eps={}, min_samples={}, n_clusters={}, sil={}".format(eps, min_samples, n_clusters, sil))
    
    plt.figure(figsize=(16, 10));
    plt.grid(True);   
    
    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))];

    for k in unique_labels:
        if k == -1:        # Black used for noise.
            col = [0, 0, 0, 1]
        else:
            col = colors[k]

        xy = X[db.labels_ == k]
        plt.plot(xy.iloc[:, 0], xy.iloc[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=14);


    plt.title('');
    plt.title("DBSCAN (eps={}, min_samples={}, n_clusters = {:d}, sil={:.3f})".format(eps, min_samples, n_clusters, sil), fontsize=20);
    plt.xlabel('Annual Income (K)', fontsize=22);
    plt.ylabel('Spending Score', fontsize=22);
    plt.xticks(fontsize=18);
    plt.yticks(fontsize=18);
    #plt.savefig('out/mall-dbscan-auto-{}-{}.png'.format(eps, min_samples), transparent=False);
    

epss = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
min_samples = range(1, 10)

for prod in list(itertools.product(epss, min_samples)):
    do_dbscan(X, prod[0], prod[1])

# Hierarchical (Agglomerative)

In [13]:
agg = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
agg.fit(JustLux)

MemoryError: 

In [ ]:
agg.labels_

In [ ]:
silhouette_score(JustLux, agg.labels_)

In [ ]:
plt.style.use('default');

plt.figure(figsize=(16, 10));
plt.grid(True);

plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=200, c=agg.labels_);
plt.title("Agglomerative", fontsize=20);
plt.xlabel('Annual Income (K)', fontsize=22);
plt.ylabel('Spending Score', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
#plt.savefig('out/mall-agg-5.png', transparent=False);

## Dendograms, Linkages, and Distance Metrics

Note that scikit learn does not itself currently support plotting dendograms. Instead, we'll use SciPy's `cluster.hierarchy` module to re-do the clustering, and plot the dendogram.

## Distance Metrics

In [ ]:
def plot_agg(X, linkage, metric):
    aggl = scipy.cluster.hierarchy.linkage(X, method=linkage, metric=metric)
    
    labels = scipy.cluster.hierarchy.fcluster(aggl, 5, criterion="maxclust")
    
    sil = 0
    n = len(set(labels))
    if n > 1:
        sil = silhouette_score(X , labels, metric=metric)
    print("Linkage={}, Metric={}, Clusters={}, Silhouette={:.3}".format(linkage, metric, n, sil))
    
    # Plot the dendogram
    plt.figure(figsize=(12, 5))  
    plt.title("Mall Dendogram (Linkage={}, Distance={}, N={}, Sil={:.3f})".format(linkage, metric, n, sil))  
    dend = scipy.cluster.hierarchy.dendrogram(aggl) 
    #plt.savefig('out/mall-agg-5-auto-dendogram-{}-{}.png'.format(linkage, metric), transparent=False);
    
    
    # Plot the points
    plt.style.use('default');
    plt.figure(figsize=(16, 10));
    plt.grid(True);

    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=200, c=labels);
    plt.title("Mall Agglomerative (Linkage={}, Distance={}, N={}, Sil={:.3f})".format(linkage, metric, n, sil), fontsize=20);
    plt.xlabel('Annual Income (K)', fontsize=22);
    plt.ylabel('Spending Score', fontsize=22);
    plt.xticks(fontsize=18);
    plt.yticks(fontsize=18);
    #plt.savefig('out/mall-agg-5-auto--{}-{}.png'.format(linkage, metric), transparent=False);

    
linkages = ['complete', 'ward', 'single', 'centroid', 'average']
metrics = ['euclidean', 'minkowski', 'cityblock', 'cosine', 'correlation', 'chebyshev', 'canberra', 'mahalanobis']

for prod in list(itertools.product(linkages, metrics)):
    
    # Some combos are not allowed
    if (prod[0] in ['ward', 'centroid']) and prod[1] != 'euclidean':
        continue
        
    plot_agg(X, prod[0], prod[1])

# Gaussian Mixture Models (GMM)

In [ ]:
gmm = GaussianMixture(n_components=5, covariance_type='full').fit(X)

labels = gmm.predict(X)
silhouette_score(X, labels)

In [ ]:
# From:
# https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_covariances.html
def make_ellipses(gmm, ax):
    for n, label in enumerate(set(labels)):
        if gmm.covariance_type == 'full':
            covariances = gmm.covariances_[n][:2, :2]
        elif gmm.covariance_type == 'tied':
            covariances = gmm.covariances_[:2, :2]
        elif gmm.covariance_type == 'diag':
            covariances = np.diag(gmm.covariances_[n][:2])
        elif gmm.covariance_type == 'spherical':
            covariances = np.eye(gmm.means_.shape[1]) * gmm.covariances_[n]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        ell = mpl.patches.Ellipse(gmm.means_[n, :2], v[0], v[1],
                                  180 + angle, color='grey')
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.2)
        ax.add_artist(ell)
        ax.set_aspect('equal', 'datalim')


plt.style.use('default');

plt.figure(figsize=(16, 10));
plt.grid(True);

plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=200, c=labels);
make_ellipses(gmm, plt.gca())
plt.title("GMM", fontsize=20);
plt.xlabel('Annual Income (K)', fontsize=22);
plt.ylabel('Spending Score', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
plt.savefig('out/mall-gmm-5.png', transparent=False);

# Categorical Data

How do we deal with categorical data? Standard K-Means and Standard Euclidean distance don't work. 

We have a number of options:

- Create dummy variables
- Run PCA first
- Use a categorical-aware distance metric, like Gower
- Use a categorical-aware algorithm, like k-mediods/k-modes

In [ ]:
scaler = StandardScaler()

X1 = df.copy()

# Create bins of Age
X1['AgeBin'] = pd.cut(X1['Age'], [0, 30, 55, 100])
X1 = X1.drop(['CustomerID', 'Age'], axis=1)

features = ['AnnualIncome', 'SpendingScore']
X1[features] = scaler.fit_transform(X1[features])

X1.info()
X1.describe().transpose()
X1.head(n=20)

# Create Dummy Variables

In [ ]:
# TODO

## Running PCA First

In [ ]:
# TODO

# Gower Distance

Unfortunately, and strangely, sci-kit learn doesn't yet have an implementation of the Gower distance metric. And I coldn't find any package that did. However, I did find some random guy's code that implemented the metric, and it seems to work well. Kudos to him!

In [ ]:
# From: https://sourceforge.net/projects/gower-distance-4python/

import numpy as np
import pandas as pd
from scipy.spatial import distance 
from sklearn.utils import validation
from sklearn.metrics import pairwise
from scipy.sparse import issparse


def check_pairwise_arrays(X, Y, precomputed=False, dtype=None):
    X, Y, dtype_float = pairwise._return_float_dtype(X, Y)

    warn_on_dtype = dtype is not None
    estimator = 'check_pairwise_arrays'
    if dtype is None:
        dtype = dtype_float

    if Y is X or Y is None:
        X = Y = validation.check_array(X, accept_sparse='csr', dtype=dtype,
                            warn_on_dtype=warn_on_dtype, estimator=estimator)
    else:
        X = validation.check_array(X, accept_sparse='csr', dtype=dtype,
                        warn_on_dtype=warn_on_dtype, estimator=estimator)
        Y = validation.check_array(Y, accept_sparse='csr', dtype=dtype,
                        warn_on_dtype=warn_on_dtype, estimator=estimator)

    if precomputed:
        if X.shape[1] != Y.shape[0]:
            raise ValueError("Precomputed metric requires shape "
                             "(n_queries, n_indexed). Got (%d, %d) "
                             "for %d indexed." %
                             (X.shape[0], X.shape[1], Y.shape[0]))
    elif X.shape[1] != Y.shape[1]:
        raise ValueError("Incompatible dimension for X and Y matrices: "
                         "X.shape[1] == %d while Y.shape[1] == %d" % (
                             X.shape[1], Y.shape[1]))

    return X, Y


# Vectorized Version
def gower_distances(X, Y=None, feature_weight=None, categorical_features=None):
    """Computes the gower distances between X and Y

    Gower is a similarity measure for categorical, boolean and numerical mixed
    data.

    
    Parameters
    ----------
    X : array-like, or pandas.DataFrame, shape (n_samples, n_features)

    Y : array-like, or pandas.DataFrame, shape (n_samples, n_features)

    feature_weight :  array-like, shape (n_features)
        According the Gower formula, feature_weight is an attribute weight.

    categorical_features: array-like, shape (n_features)
        Indicates with True/False whether a column is a categorical attribute.
        This is useful when categorical atributes are represented as integer
        values. Categorical ordinal attributes are treated as numeric, and must
        be marked as false.
        
        Alternatively, the categorical_features array can be represented only
        with the numerical indexes of the categorical attribtes.

    Returns
    -------
    similarities : ndarray, shape (n_samples, n_samples)

    Notes
    ------
    The non-numeric features, and numeric feature ranges are determined from X and not Y.
    No support for sparse matrices.

    """
    
    if issparse(X) or issparse(Y):
        raise TypeError("Sparse matrices are not supported for gower distance")
        
    y_none = Y is None
    
    
    # It is necessary to convert to ndarray in advance to define the dtype
    if not isinstance(X, np.ndarray):
        X = np.asarray(X)

    array_type = np.object
    # this is necessary as strangelly the validator is rejecting numeric
    # arrays with NaN
    if  np.issubdtype(X.dtype, np.number) and (np.isfinite(X.sum()) or np.isfinite(X).all()):
        array_type = type(np.zeros(1,X.dtype).flat[0])
    
    X, Y = check_pairwise_arrays(X, Y, precomputed=False, dtype=array_type)
    
    n_rows, n_cols = X.shape
    
    if categorical_features is None:
        categorical_features = np.zeros(n_cols, dtype=bool)
        for col in range(n_cols):
            # In numerical columns, None is converted to NaN,
            # and the type of NaN is recognized as a number subtype
            if not np.issubdtype(type(X[0, col]), np.number):
                categorical_features[col]=True
    else:          
        categorical_features = np.array(categorical_features)
    
    
    #if categorical_features.dtype == np.int32:
    if np.issubdtype(categorical_features.dtype, np.int):
        new_categorical_features = np.zeros(n_cols, dtype=bool)
        new_categorical_features[categorical_features] = True
        categorical_features = new_categorical_features
    
    print(categorical_features)
  
    # Categorical columns
    X_cat =  X[:,categorical_features]
    
    # Numerical columns
    X_num = X[:,np.logical_not(categorical_features)]
    ranges_of_numeric = None
    max_of_numeric = None
    
        
    # Calculates the normalized ranges and max values of numeric values
    _ ,num_cols=X_num.shape
    ranges_of_numeric = np.zeros(num_cols)
    max_of_numeric = np.zeros(num_cols)
    for col in range(num_cols):
        col_array = X_num[:, col].astype(np.float32) 
        max = np.nanmax(col_array)
        min = np.nanmin(col_array)
     
        if np.isnan(max):
            max = 0.0
        if np.isnan(min):
            min = 0.0
        max_of_numeric[col] = max
        ranges_of_numeric[col] = (1 - min / max) if (max != 0) else 0.0


    # This is to normalize the numeric values between 0 and 1.
    X_num = np.divide(X_num ,max_of_numeric,out=np.zeros_like(X_num), where=max_of_numeric!=0)

    
    if feature_weight is None:
        feature_weight = np.ones(n_cols)
        
    feature_weight_cat=feature_weight[categorical_features]
    feature_weight_num=feature_weight[np.logical_not(categorical_features)]
    
    
    y_n_rows, _ = Y.shape
    
    dm = np.zeros((n_rows, y_n_rows), dtype=np.float32)
        
    feature_weight_sum = feature_weight.sum()

    Y_cat=None
    Y_num=None
    
    if not y_none:
        Y_cat = Y[:,categorical_features]
        Y_num = Y[:,np.logical_not(categorical_features)]
        # This is to normalize the numeric values between 0 and 1.
        Y_num = np.divide(Y_num ,max_of_numeric,out=np.zeros_like(Y_num), where=max_of_numeric!=0)
    else:
        Y_cat=X_cat
        Y_num = X_num
        
    for i in range(n_rows):
        j_start= i
        
        # for non square results
        if n_rows != y_n_rows:
            j_start = 0

      
        Y_cat[j_start:n_rows,:]
        Y_num[j_start:n_rows,:]
        result= _gower_distance_row(X_cat[i,:], X_num[i,:],Y_cat[j_start:n_rows,:],
                                    Y_num[j_start:n_rows,:],feature_weight_cat,feature_weight_num,
                                    feature_weight_sum,categorical_features,ranges_of_numeric,
                                    max_of_numeric) 
        dm[i,j_start:]=result
        dm[i:,j_start]=result
        

    return dm


def _gower_distance_row(xi_cat,xi_num,xj_cat,xj_num,feature_weight_cat,feature_weight_num,
                        feature_weight_sum,categorical_features,ranges_of_numeric,max_of_numeric ):
    # categorical columns
    sij_cat = np.where(xi_cat == xj_cat,np.zeros_like(xi_cat),np.ones_like(xi_cat))
    sum_cat = np.multiply(feature_weight_cat,sij_cat).sum(axis=1) 

    # numerical columns
    abs_delta=np.absolute( xi_num-xj_num)
    sij_num=np.divide(abs_delta, ranges_of_numeric, out=np.zeros_like(abs_delta), where=ranges_of_numeric!=0)

    sum_num = np.multiply(feature_weight_num,sij_num).sum(axis=1)
    sums= np.add(sum_cat,sum_num)
    sum_sij = np.divide(sums,feature_weight_sum)
    return sum_sij


In [ ]:
D = gower_distances(X1)
aggl = scipy.cluster.hierarchy.linkage(D)
labels = scipy.cluster.hierarchy.fcluster(aggl, 6, criterion="maxclust")
n = len(set(labels))

# Plot the dendogram
plt.figure(figsize=(12, 5));
plt.title("Mall Dendogram (N={})".format(n)); 
dend = scipy.cluster.hierarchy.dendrogram(aggl) 
plt.savefig('out/mall-agg-5-gower-dendogram.png', transparent=False);

In [ ]:
def describe_clusters(X, labels):
    X2 = X.copy()
    X2['ClusterID'] = labels
    print('\nCluster sizes:')
    print(X2.groupby('ClusterID').size())
    
    
    print('\nCluster stats:')
    print(X2.groupby('ClusterID').mean())
    print(X2.groupby('ClusterID').agg(lambda x: x.value_counts().index[0]))
    
    #display(X2.groupby('ClusterID').describe(include='all').transpose())

describe_clusters(X1, labels)

# K-Modes
`kmodes` is an experimental library in Python

In [ ]:
kmodes = KModes(n_clusters=6, init='Huang', n_init=100, verbose=0)
kmodes.fit(X1)

In [ ]:
kmodes.labels_

In [ ]:
# Print the cluster centroids
print(kmodes.cluster_centroids_)

kmodes.labels_

describe_clusters(X1, kmodes.labels_)

plt.style.use('default');
fig = plt.figure(figsize=(16, 10));
plt.grid(True);
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X1['AnnualIncome'], X1['SpendingScore'], X1['Male'], s=200, c=kmodes.labels_);
plt.title("KModes", fontsize=20);
plt.xlabel('Annual Income (K)', fontsize=22);
plt.ylabel('Spending Score', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
plt.savefig('out/mall-kmodes-6-3d.png', transparent=False);

# Distance Metrics

In [ ]:
import scipy.spatial.distance as dist
import numpy as np

# Prepare 3 vectors (data points) of 2 dimensions
points = pd.DataFrame(data={'id':['A', 'B', 'C'], 'dim1':[1, 2, 5], 'dim2':[1, 1, 5]})
display(points.head())

for combo in itertools.combinations(points['id'], 2):
    v1 = points[points['id'] == combo[0]][['dim1', 'dim2']]
    v2 = points[points['id'] == combo[1]][['dim1', 'dim2']]
    
    print('\nPoints {} and {}:'.format(combo[0], combo[1]))
    print('Euclidean distance: {:.2f}'.format(dist.euclidean(v1, v2)))
    print('Manhattan distance: {:.2f}'.format(dist.cityblock(v1, v2)))
    print('Chebyshev distance: {:.2f}'.format(dist.chebyshev(v1, v2)))
    print('Canberra distance: {:.2f}'.format(dist.canberra(v1, v2)))
    print('Cosine distance: {:.2f}'.format(dist.cosine(v1, v2)))
    
    
plt.style.use('default');

plt.figure(figsize=(10, 10));
plt.grid(True);

plt.scatter(points['dim1'], points['dim2'], s=6000, marker='o', edgecolors='black');
#plt.title("Distances", fontsize=20);
plt.xlabel('Feature 1', fontsize=32);
plt.ylabel('Feature 2', fontsize=32);
plt.xticks(fontsize=26);
plt.yticks(fontsize=26);
plt.xlim(left=0, right=6);
plt.ylim(bottom=0, top=6);

for i, txt in enumerate(points['id']):
    plt.text(x=points['dim1'].iloc[i], y=points['dim2'].iloc[i], s=txt, fontsize=32, 
             horizontalalignment='center', verticalalignment='center', color='white');
    


plt.savefig('out/distances.png', transparent=False);

## Categorical Features

In [ ]:
points = pd.DataFrame(data={'id':['A', 'B', 'C'], 'dim1':['young', 'young', 'old'], 
                            'dim2':['male', 'male', 'female'], 'dim3':['rural', 'city', 'city'],
                            'dim4':['student', 'student', 'prof'], 'dim5':['low', 'low', 'high'],})
display(points.head())

for combo in itertools.combinations(points['id'], 2):
    v1 = points[points['id'] == combo[0]][['dim1', 'dim2', 'dim3', 'dim4', 'dim5']]
    v2 = points[points['id'] == combo[1]][['dim1', 'dim2', 'dim3', 'dim4', 'dim5']]
    
    print('\nPoints {} and {}:'.format(combo[0], combo[1]))
    print('Hamming distance: {:.2f}'.format(dist.hamming(v1, v2)))